This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Find the shortest path between two nodes in a graph.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is this a directional graph?
    * Yes
* Could the graph have cycles?
    * Yes
    * Note: If the answer were no, this would be a DAG.  
        * DAGs can be solved with a [topological sort](http://www.geeksforgeeks.org/shortest-path-for-directed-acyclic-graphs/)
* Are the edges weighted?
    * Yes
    * Note: If the edges were not weighted, we could do a BFS
* Are the edges all non-negative numbers?
    * Yes
    * Note: Graphs with negative edges can be done with Bellman-Ford
        * Graphs with negative cost cycles do not have a defined shortest path
* Do we have to check for non-negative edges?
    * No
* Can we assume this is a connected graph?
    * Yes
* Can we assume the inputs are valid?
    * No
* Can we assume we already have a graph class?
    * Yes
* Can we assume we already have a priority queue class?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

The constraints state we don't have to check for negative edges, so we test with the general case.

<pre>
graph.add_edge('a', 'b', weight=5)
graph.add_edge('a', 'c', weight=3)
graph.add_edge('a', 'e', weight=2)
graph.add_edge('b', 'd', weight=2)
graph.add_edge('c', 'b', weight=1)
graph.add_edge('c', 'd', weight=1)
graph.add_edge('d', 'a', weight=1)
graph.add_edge('d', 'g', weight=2)
graph.add_edge('d', 'h', weight=1)
graph.add_edge('e', 'a', weight=1)
graph.add_edge('e', 'h', weight=4)
graph.add_edge('e', 'i', weight=7)
graph.add_edge('f', 'b', weight=3)
graph.add_edge('f', 'g', weight=1)
graph.add_edge('g', 'c', weight=3)
graph.add_edge('g', 'i', weight=2)
graph.add_edge('h', 'c', weight=2)
graph.add_edge('h', 'f', weight=2)
graph.add_edge('h', 'g', weight=2)
shortest_path = ShortestPath(graph)
result = shortest_path.find_shortest_path('a', 'i')
assert_equal(result, ['a', 'c', 'd', 'g', 'i'])
assert_equal(shortest_path.path_weight['i'], 8)
</pre>

## Algorithm

Refer to the [Solution Notebook](http://nbviewer.jupyter.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/graph_shortest_path/graph_shortest_path_solution.ipynb).  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

**Solution:** Dijkstra's algorithm. Keep track of the path via a 'parent' attr for each node, along with its weight so far. To recover the path, backtrack through the graph via the parent attribute.

## Code

In [1]:
import numpy as np

In [2]:
# %load ../../arrays_strings/priority_queue/priority_queue.py
import sys


class PriorityQueueNode(object):

    def __init__(self, obj, key):
        self.obj = obj
        self.key = key

    def __repr__(self):
        return str(self.obj) + ': ' + str(self.key)


class PriorityQueue(object):

    def __init__(self):
        self.array = []

    def __len__(self):
        return len(self.array)

    def insert(self, node):
        self.array.append(node)
        return self.array[-1]

    def extract_min(self):
        if not self.array:
            return None
        minimum = sys.maxsize
        for index, node in enumerate(self.array):
            if node.key < minimum:
                minimum = node.key
                minimum_index = index
        return self.array.pop(minimum_index)

    def decrease_key(self, obj, new_key):
        for node in self.array:
            if node.obj is obj:
                node.key = new_key
                return node
        return None

In [3]:
# %load ../graph/graph.py
from enum import Enum  # Python 2 users: Run pip install enum34


class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2


class Node:

    def __init__(self, key):
        self.key = key
        self.visit_state = State.unvisited
        self.incoming_edges = 0
        self.adj_nodes = {}  # Key = key, val = Node
        self.adj_weights = {}  # Key = key, val = weight

    def __repr__(self):
        return str(self.key)

    def __lt__(self, other):
        return self.key < other.key

    def add_neighbor(self, neighbor, weight=0):
        if neighbor is None or weight is None:
            raise TypeError('neighbor or weight cannot be None')
        neighbor.incoming_edges += 1
        self.adj_weights[neighbor.key] = weight
        self.adj_nodes[neighbor.key] = neighbor

    def remove_neighbor(self, neighbor):
        if neighbor is None:
            raise TypeError('neighbor cannot be None')
        if neighbor.key not in self.adj_nodes:
            raise KeyError('neighbor not found')
        neighbor.incoming_edges -= 1
        del self.adj_weights[neighbor.key]
        del self.adj_nodes[neighbor.key]


class Graph:

    def __init__(self):
        self.nodes = {}  # Key = key, val = Node

    def add_node(self, key):
        if key is None:
            raise TypeError('key cannot be None')
        if key not in self.nodes:
            self.nodes[key] = Node(key)
        return self.nodes[key]

    def add_edge(self, source_key, dest_key, weight=0):
        if source_key is None or dest_key is None:
            raise KeyError('Invalid key')
        if source_key not in self.nodes:
            self.add_node(source_key)
        if dest_key not in self.nodes:
            self.add_node(dest_key)
        self.nodes[source_key].add_neighbor(self.nodes[dest_key], weight)

    def add_undirected_edge(self, src_key, dst_key, weight=0):
        if src_key is None or dst_key is None:
            raise TypeError('key cannot be None')
        self.add_edge(src_key, dst_key, weight)
        self.add_edge(dst_key, src_key, weight)

In [4]:
class ShortestPath(object):

    def __init__(self, graph):
        self.graph = graph
        self.path_weight = {}  # key is key, value is integer representing the path weight
        
    def find_shortest_path(self, start_node_key, end_node_key):
        unvisited = set(self.graph.nodes.values())
        priority_q = PriorityQueue()
        for node in self.graph.nodes.values():
            node.dist = np.inf
            priority_q.insert(PriorityQueueNode(node, node.dist))
            self.path_weight[node.key] = node.dist

        start_node = self.graph.nodes[start_node_key]
        start_node.dist = 0
        start_node.parent = None
        priority_q.decrease_key(start_node, start_node.dist)
        
        while len(unvisited):
            current = priority_q.extract_min().obj
            if current.key == end_node_key:
                # found the shortest path...
                # backtrack and return the path.
                return self.backtrack(current, start_node)
            
            for neighbor_key in current.adj_nodes:
                neighbor, weight = current.adj_nodes[neighbor_key], current.adj_weights[neighbor_key]
                # possibly update distance if it's shorter
                if neighbor.dist > current.dist + weight:
                    neighbor.dist = current.dist + weight
                    neighbor.parent = current
                    # update weight in priority_queue
                    priority_q.decrease_key(neighbor, neighbor.dist)
                    self.path_weight[neighbor_key] = neighbor.dist
            unvisited.remove(current)

    def backtrack(self, end_node, start_node):
        path = []
        current = end_node
        while current is not None:
            path.append(current.key)
            current = current.parent
        path.reverse()
        return path

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [5]:
# %load test_shortest_path.py
from nose.tools import assert_equal


class TestShortestPath(object):

    def test_shortest_path(self):
        graph = Graph()
        graph.add_edge('a', 'b', weight=5)
        graph.add_edge('a', 'c', weight=3)
        graph.add_edge('a', 'e', weight=2)
        graph.add_edge('b', 'd', weight=2)
        graph.add_edge('c', 'b', weight=1)
        graph.add_edge('c', 'd', weight=1)
        graph.add_edge('d', 'a', weight=1)
        graph.add_edge('d', 'g', weight=2)
        graph.add_edge('d', 'h', weight=1)
        graph.add_edge('e', 'a', weight=1)
        graph.add_edge('e', 'h', weight=4)
        graph.add_edge('e', 'i', weight=7)
        graph.add_edge('f', 'b', weight=3)
        graph.add_edge('f', 'g', weight=1)
        graph.add_edge('g', 'c', weight=3)
        graph.add_edge('g', 'i', weight=2)
        graph.add_edge('h', 'c', weight=2)
        graph.add_edge('h', 'f', weight=2)
        graph.add_edge('h', 'g', weight=2)
        shortest_path = ShortestPath(graph)
        result = shortest_path.find_shortest_path('a', 'i')
        assert_equal(result, ['a', 'c', 'd', 'g', 'i'])
        assert_equal(shortest_path.path_weight['i'], 8)

        print('Success: test_shortest_path')


def main():
    test = TestShortestPath()
    test.test_shortest_path()


if __name__ == '__main__':
    main()

Success: test_shortest_path


## Solution Notebook

Review the [Solution Notebook](https://github.com/donnemartin/interactive-coding-challenges/graphs_trees/graph_shortest_path/graph_shortest_path_solution.ipynb) for a discussion on algorithms and code solutions.